In [1]:
import os
import vllm
import json
from easysteer.hidden_states import get_all_hidden_states
from vllm import LLM,SamplingParams

from easysteer.steer import extract_pca_control_vector,StatisticalControlVector


os.environ["CUDA_VISIBLE_DEVICES"] = "2"
os.environ["VLLM_USE_V1"] = "0"

# file_path = "../../temp/test.jsonl" #MATH-500
file_path = "../../temp/test.jsonl" #GSM8K

model_path = "/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/"

# vector_path = "vectors/thinking_switch_pca_MATH-500.gguf" #MATH-500
vector_path = "MATH500.gguf"

num_question = 4

/home/meixinyu/anaconda3/envs/easysteer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 07-13 17:53:54 [__init__.py:244] Automatically detected platform cuda.


In [2]:
problem_list = []

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        line_count = 0
        for line in f:
            if line_count >= num_question:
                break
            stripped_line = line.strip()
            if not stripped_line:
                continue
                
            try:
                data = json.loads(stripped_line)
                if "problem" in data:
                    problem_list.append(data["problem"])
                    line_count += 1
                elif "question" in data:
                    problem_list.append(data["question"])
                    line_count += 1
            except json.JSONDecodeError:
                print(f"skip: {line[:50]}...")
                continue

except Exception as e:
    print(f"{str(e)}")
# problem_list = ["Find the roots of $(x - 3)^3 + (x -7)^3 = (2x - 10)^3.$","A regular hexagon can be divided into six equilateral triangles. If the perimeter of one of the triangles is 21 inches, what is the perimeter, in inches, of the regular hexagon?",""]

In [3]:
texts_fast = []
texts_slow = []

for prob in problem_list:
    texts_fast.append("<|User|>Return your final response within \\boxed{}."+prob+"\n<|Assistant|><think>\nTo")

for prob in problem_list:
    texts_slow.append("<|User|>Return your final response within \\boxed{}."+prob+"\n<|Assistant|><think>\nAlright")


llm = LLM(model=model_path,task="generate",tensor_parallel_size=1,enforce_eager=True)

answers_fast = llm.generate(
    texts_fast,
    SamplingParams(
        temperature=0,
        max_tokens=4096,
        skip_special_tokens=False,
    ),
)

answers_fast = [answer.outputs[0].text for answer in answers_fast]
qa_pairs_fast = [texts_fast[i] + answers_fast[i] for i in range(len(texts_fast))]




INFO 07-13 17:54:02 [config.py:1472] Using max model len 131072
WARNING 07-13 17:54:02 [arg_utils.py:1566] Chunked prefill is enabled by default for models with max_model_len > 32K. Chunked prefill might not work with some features or models. If you encounter any issues, please disable by launching with --enable-chunked-prefill=False.


2025-07-13 17:54:03,001	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 07-13 17:54:03 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=2048.
WARNING 07-13 17:54:03 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-13 17:54:03 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='a

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.07s/it]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.07s/it]


INFO 07-13 17:54:06 [default_loader.py:272] Loading weights took 1.21 seconds
INFO 07-13 17:54:06 [model_runner.py:1255] Model loading took 3.3461 GiB and 1.363570 seconds


INFO 07-13 17:54:07 [worker.py:295] Memory profiling takes 0.64 seconds
INFO 07-13 17:54:07 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 07-13 17:54:07 [worker.py:295] model weights take 3.35GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.41GiB; the rest of the memory reserved for KV Cache is 37.85GiB.
INFO 07-13 17:54:07 [executor_base.py:115] # cuda blocks: 88594, # CPU blocks: 9362
INFO 07-13 17:54:07 [executor_base.py:120] Maximum concurrency for 131072 tokens per request: 10.81x
INFO 07-13 17:54:11 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 5.00 seconds


Processed prompts: 100%|██████████| 4/4 [00:17<00:00,  4.40s/it, est. speed input: 17.49 toks/s, output: 126.44 toks/s]


In [4]:
answers_slow = llm.generate(
    texts_slow,
    SamplingParams(
        temperature=0,
        max_tokens=4096,
        skip_special_tokens=False,
    ),
)
# texts=texts_fast+texts_slow


answers_slow = [answer.outputs[0].text for answer in answers_slow]
qa_pairs_slow = [texts_slow[i] + answers_slow[i] for i in range(len(texts_slow))]



Processed prompts: 100%|██████████| 4/4 [01:24<00:00, 21.14s/it, est. speed input: 3.64 toks/s, output: 101.94 toks/s]


In [5]:
llm = LLM(model=model_path,task="reward",tensor_parallel_size=1)

hidden_states_fast, _= get_all_hidden_states(llm, qa_pairs_fast)
hidden_states_slow, _= get_all_hidden_states(llm, qa_pairs_slow)

INFO 07-13 17:55:55 [config.py:1472] Using max model len 16384
INFO 07-13 17:55:55 [config.py:4615] Only "last" pooling supports chunked prefill and prefix caching; disabling both.
INFO 07-13 17:55:55 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_propertie

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.08it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.07it/s]


INFO 07-13 17:55:58 [default_loader.py:272] Loading weights took 1.05 seconds


INFO 07-13 17:55:59 [model_runner.py:1255] Model loading took 2.8793 GiB and 1.101507 seconds


Processed prompts: 100%|██████████| 4/4 [00:01<00:00,  3.60it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


In [6]:
from transformers import AutoTokenizer

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/")

# The newline token suffix in tokenizer vocabulary
target_suffix = "ĊĊ"  # "\n\n" is tokenized as "ĊĊ"

all_hidden_states=[]
# Process each QA pair to find newline positions
fast_token = []
fast_positions = []
for i,answer in enumerate(qa_pairs_fast):
    # Tokenize the QA pair
    tokens = tokenizer.tokenize(answer, add_special_tokens=True)
    fast_token.append(tokens)
    
    # Find all positions of "ĊĊ" in the tokens
    # These represent potential paragraph breaks in the text
    positions = [
        i for i, token in enumerate(tokens) 
        if isinstance(token, str) and token.endswith(target_suffix)
    ]
    second_position = positions[1] if len(positions) > 1 else None
    fast_positions.append(second_position)
    print(len(hidden_states_fast[i][0]))
    
    all_hidden_states.append([
        [hidden_states_fast[i][layer][second_position]]  # 注意这里多加了一层方括号
        for layer in range(28)
    ])
    # all_hidden_states.append([
    #     hidden_states_slow[i][layer][second_position]
    #     for layer in range(19, 28)
    # ])



416
992
714
408


In [7]:
# Process slow-thinking responses to find truncation positions
slow_tokens = []
slow_truncation_positions = []
for i, answer in enumerate(qa_pairs_slow):
    # Tokenize the response
    tokens = tokenizer.tokenize(answer, add_special_tokens=True)
    slow_tokens.append(tokens)
    
    # Get the corresponding slow-thinking second position
    fast_second_pos = fast_positions[i]
    
    # if fast_second_pos is None:
    #     slow_truncation_positions.append(None)
    #     continue
    
    # Find all positions of "ĊĊ" in slow-thinking response
    slow_positions = [
        j for j, token in enumerate(tokens) 
        if isinstance(token, str) and token.endswith(target_suffix)
    ]
    
    # if not slow_positions:
    #     slow_truncation_positions.append(None)
    #     continue
    
    # Find the position with nearest length to fast-thinking response
    closest_position = min(slow_positions, key=lambda x: abs(x - fast_second_pos))
    slow_truncation_positions.append(closest_position)
    
    all_hidden_states.append([
        [hidden_states_slow[i][layer][closest_position]]  # 注意这里多加了一层方括号
        for layer in range(28)
    ])

In [8]:

control_vector = extract_pca_control_vector(
    all_hidden_states=all_hidden_states,
    positive_indices=list(range(num_question)), 
    negative_indices=list(range(num_question,2*num_question)),
    model_type="qwen2.5",
    method="center",
    token_pos=-1,
    normalize=False
)

Computing PCA directions: 100%|██████████| 28/28 [00:00<00:00, 1267.02it/s]


In [9]:
control_vector.export_gguf(vector_path)
control_vector = StatisticalControlVector.import_gguf(vector_path)
print(control_vector)

StatisticalControlVector(model_type='qwen2.5', method='pca', directions={0: memmap([-0.0454458 ,  0.01361481,  0.01698131, ...,  0.0365207 ,
         0.0359213 ,  0.00463693], shape=(1536,), dtype=float32), 1: memmap([-0.03687522,  0.01788453, -0.00667728, ...,  0.03739358,
         0.01404337,  0.0170301 ], shape=(1536,), dtype=float32), 2: memmap([-0.03864472, -0.00628723, -0.02879711, ...,  0.02464086,
         0.02684358,  0.01252756], shape=(1536,), dtype=float32), 3: memmap([ 0.02786415,  0.00761234,  0.01361886, ..., -0.00188065,
        -0.04521464, -0.01735996], shape=(1536,), dtype=float32), 4: memmap([ 0.03038092, -0.0011958 ,  0.00820926, ...,  0.01588251,
        -0.02874249, -0.01130663], shape=(1536,), dtype=float32), 5: memmap([-0.04012294, -0.01570465, -0.00127555, ..., -0.0050538 ,
         0.02423198,  0.03660234], shape=(1536,), dtype=float32), 6: memmap([-0.04255249, -0.02186495,  0.01580931, ..., -0.00768411,
         0.01432822,  0.01873141], shape=(1536,), dtype

: 